In [2]:
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# from PIL import Image

# Define paths to the finger vein dataset directory
data_dir = "FVData"
train_dir= "Data/FVData/train"
# val_dir = "FVData/validation"
test_dir = "Data/FVData/test"

# Split the dataset into training and validation sets
# train_dir, val_dir = train_test_split(os.listdir(data_dir), test_size=0.2, random_state=42)

# Image preprocessing function
def preprocess_image(img):
    # Customize this function based on your preprocessing requirements
    return img / 255.0  # Normalize to the range [0, 1]

# from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Additional checks
print("Number of classes (training):", len(train_generator.class_indices))
print("Number of classes (validation):", len(validation_generator.class_indices))
print("Number of classes (test):", len(test_generator.class_indices))

# print("Number of images in validation set:", validation_generator.samples)

# Display a few images from the training generator
print("Sample images from the validation set:")
for _ in range(3):
    batch = validation_generator.next()
    print(batch[0].shape, batch[1].shape)



# print(len(validation_generator))
import matplotlib.pyplot as plt

# Display a few images from the training generator
def show_images(generator, num_images=5):
    class_labels = list(generator.class_indices.keys())
    batch = generator.next()
    images, labels = batch

    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i])
        plt.title(f"Class: {class_labels[np.argmax(labels[i])]}")
        plt.axis("off")

    plt.show()

# Display images from the training generator
show_images(train_generator, num_images=5)

# Display images from the validation generator
show_images(validation_generator, num_images=5)

#Display images from test generator
show_images(test_generator,num_images=5)

# Model definition
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    layers.BatchNormalization(), # Additional dense layer
    # tf.keras.layers.Dropout(0.5),  # Dropout layer for regularization
    Dense(128, activation="relu"),
    layers.BatchNormalization(),
    Dense(64, activation="relu"),
    layers.BatchNormalization(),
    Dense(32, activation="relu"),
    layers.BatchNormalization(), # Additional dense layer
    Dense(len(train_generator.class_indices), activation="softmax")
])

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Set up EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Fine-tuning: Unfreeze the last 20 layers of the base model
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=1,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)
# Evaluate the model on the test set
test_results = model.evaluate(test_generator, steps=len(test_generator))
print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])
# Example usage for predicting an unknown finger vein image
# unknown_finger_path = "SCUT FV Dataset/session1_ROI/session1_ROI/1/1_1_0_0_1.bmp"
# predicted_class = model.predict(train_datagen.flow(np.expand_dims(preprocess_image(cv2.imread(unknown_finger_path)), axis=0)))
# print(f"Predicted class: {predicted_class}")
# unknown_finger_path = "sample_dataset/short_sample/3/3_1_0_0_1.bmp"
# predicted_class_probabilities = model.predict(train_datagen.flow(np.expand_dims(preprocess_image(cv2.imread(unknown_finger_path)), axis=0)))
# # predicted_class = np.argmax(predicted_class_probabilities, axis=1)[0]
# predicted_class_label = list(train_generator.class_indices.keys())[predicted_class]
# print(f"Predicted class: {predicted_class}")




Found 98170 images belonging to 96 classes.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'FVData/validation'

In [33]:
unknown_finger_path = "collected_final/80/80_1_0_0_1.bmp"
from tensorflow.keras.preprocessing import image

# Example usage for predicting an unknown finger vein image
# Load and preprocess the image
unknown_finger_img = image.load_img(unknown_finger_path, target_size=(224, 224))
unknown_finger_img_array = image.img_to_array(unknown_finger_img)
unknown_finger_img_array = preprocess_image(unknown_finger_img_array)
unknown_finger_img_array = np.expand_dims(unknown_finger_img_array, axis=0)

# Predict the class probabilities
predicted_class_probabilities = model.predict(unknown_finger_img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predicted_class_probabilities, axis=1)[0]

# Map the class index to the actual class label using the data generator's class_indices
class_label = list(train_generator.class_indices.keys())[predicted_class_index]

print(f"Predicted class: {class_label}")

1/1 [==============================] - 0s 33ms/step
Predicted class: 1


In [35]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\ssnfs\DataspellProjects\FVProject
